In [9]:
from skt.vault_utils import get_secrets
from skt.ye import get_spark
proxies = get_secrets('proxies')

In [10]:
import pyspark.sql.functions as F

In [11]:
import os

In [12]:
os.environ['http_proxy'] = proxies['http']
os.environ['https_proxy'] = proxies['https']

In [13]:
spark = get_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/06 19:20:23 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [8]:
tidc_hadoop_ip = "hdfs://172.27.23.141:8020"

In [15]:
tag_hadoop_path = f'{tidc_hadoop_ip}/recommendation/apollo/music/plugin/knowledge_vocab/*'

In [16]:
music_df = spark.read.json(tag_hadoop_path)

In [21]:
from skt.gcp import load_bigquery_ipython_magic, \
                    bq_to_pandas, \
                    get_bigquery_client

In [27]:
from skt.gcp import df_to_bq_table
from pyspark.sql.types import DateType
import time

In [22]:
dest_dataset = "x1112436"
partitioned_dest_table = f"music_formula"

get_bigquery_client().query(f"""
    CREATE TABLE IF NOT EXISTS {dest_dataset}.{partitioned_dest_table}
    (
        id  STRING,
        name STRING,
        type STRING    
    )
""").result()

print(f"생성된 테이블 : {dest_dataset}.{partitioned_dest_table}")

생성된 테이블 : x1112436.music_formula


In [24]:
df_to_bq_table(df=music_df,
               dataset=dest_dataset,
               table_name=partitioned_dest_table,
               mode="overwrite")

# LOAD

In [25]:
from skt.gcp import load_bigquery_ipython_magic, \
                    bq_to_pandas, \
                    get_bigquery_client

In [2]:
dest_dataset = "x1112436"
partitioned_dest_table = f"music_formula"

query = f"""

SELECT  *
FROM `skt-datahub.{dest_dataset}.{partitioned_dest_table}`
"""

In [3]:
formula_table = bq_to_pandas(query)

unsupported operand type(s) for /: 'NoneType' and 'int'
Downloading: 100%|██████████|


In [5]:
formula_keywords = list(formula_table.name.unique())

# load alias

In [75]:
tidc_hadoop_ip = 'hdfs://172.27.1.237:8020//'

In [17]:
alias_file = f'{tidc_hadoop_ip}/data/music/source/knowledge/alias/*'

In [66]:
alias = spark.read.json(alias_file)

In [61]:
#alias.printSchema()

In [67]:
alias = alias.select(
    F.col('alias.name').alias('alias_nms'),
    F.col('name')
)

In [70]:
alias = alias.withColumn('alias_nm', F.explode('alias_nms')).select('alias_nm', 'name')

In [72]:
alias = alias.distinct()

In [55]:
# dest_dataset = "x1112436"
# table_nm = f"music_formula_alias"
# get_bigquery_client().query(f"""
#     DROP TABLE IF EXISTS {dest_dataset}.{table_nm}"""
# )

In [73]:
df_to_bq_table(df=alias,
               dataset=dest_dataset,
               table_name=partitioned_dest_table,
               mode="overwrite")

# Load artist

In [85]:
artist = spark.read.json(f'{tidc_hadoop_ip}/recommendation/workspace/jaehwan/music_interact/artist_word.json')

In [86]:
dest_dataset = "x1112436"
table_nm = f"music_artist"

In [88]:
df_to_bq_table(df=artist,
               dataset=dest_dataset,
               table_name=table_nm,
               mode="overwrite")

In [89]:
artist.count()

4663248